In [1]:
from service.models.Api import SearchIndexRequest, SearchRequest, SearchResponse, SearchResponseItem
from service.search.SearchService import SearchService
from service.models.SearchConfig import IndexConfigs, IndexConfig
from service.models.Test import TestData, TestResult, TestResults, TestSearchProfile, TestStats

from dotenv import load_dotenv
import os

from typing import Dict, List

import json
from pathlib import Path

import requests

load_dotenv()

/opt/homebrew/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [2]:
mongo_db_secret = os.getenv("MONGO_DB")

index_configs = IndexConfigs(indexes=[
    IndexConfig(vector_model="all-MiniLM-L6-v2", index_name="all-MiniLM-L6-v2", vector_size=384),
    IndexConfig(vector_model="paraphrase-MiniLM-L6-v2", index_name="paraphrase-MiniLM-L6-v2", vector_size=384),
    IndexConfig(vector_model="all-distilroberta-v1", index_name="all-distilroberta-v1", vector_size=768),
    IndexConfig(vector_model="nomic-ai/nomic-embed-text-v2-moe", index_name= "nomic-embed-text-v2", vector_size=768),
    IndexConfig(vector_model="all-MiniLM-L6-v2", index_name="summary-all-MiniLM-L6-v2", vector_size=384),
    IndexConfig(vector_model="paraphrase-MiniLM-L6-v2", index_name="summary-paraphrase-MiniLM-L6-v2", vector_size=384),
    IndexConfig(vector_model="all-distilroberta-v1", index_name="summary-all-distilroberta-v1", vector_size=768),
    IndexConfig(vector_model="nomic-ai/nomic-embed-text-v2-moe", index_name= "summary-nomic-embed-text-v2", vector_size=768),
])

print("Loading search service...")
search_service : SearchService = SearchService(mongo_db_secret, index_configs, initialize=False, docker=True)
print("Search service loaded")

Loading search service...


!!!!!!!!!!!!megablocks not available, using torch.matmul instead


Search service loaded


In [3]:
queries = [
    "How does incremental hydration help improve system performance?",
    "Preventing cross-site scripting",
    "How to use Angular Dependency Injection?",
    "being reactive with resources",
    "explain the benefits of server-side and hybrid rendering",
]

In [12]:
requests : Dict[str, SearchRequest] = {}
for query in queries:
    query_key = query.replace(" ", "_").replace("?", "").replace(":", "").replace(",", "")
    for index_config in index_configs.indexes:
        search_request = SearchRequest(
            query=query,
            generate_queries=False,
            index_requests=[
                SearchIndexRequest(index_name=index_config.index_name, top_k=20, confidence=0.0)
            ],
            top_k=20,
        )
        requests[f"{query_key}-{index_config.index_name}"] = search_request
    
    # best f1 score index combo
    requests[f"{query_key}-f1"] = SearchRequest(
            query=query,
            generate_queries=False,
            index_requests=[
                SearchIndexRequest(index_name="all-MiniLM-L6-v2", top_k=3, confidence=0.3),
                SearchIndexRequest(index_name="all-distilroberta-v1", top_k=3, confidence=0.3),
                SearchIndexRequest(index_name="nomic-embed-text-v2", top_k=3, confidence=0.3),
                SearchIndexRequest(index_name="summary-all-MiniLM-L6-v2", top_k=3, confidence=0.3),
                SearchIndexRequest(index_name="summary-all-distilroberta-v1", top_k=3,confidence=0.3),
                SearchIndexRequest(index_name="summary-nomic-embed-text-v2", top_k=3, confidence=0.6),
            ],
            top_k=3,
        )
    
    
    # best mrr score index combo
    requests[f"{query_key}-mrr"] = SearchRequest(
        query=query,
        generate_queries=False,
        index_requests=[
            SearchIndexRequest(index_name="all-MiniLM-L6-v2", top_k=3, confidence=0.3),
            SearchIndexRequest(index_name="all-distilroberta-v1", top_k=3, confidence=0.3),
            SearchIndexRequest(index_name="nomic-embed-text-v2", top_k=3, confidence=0.3),
            SearchIndexRequest(index_name="summary-all-MiniLM-L6-v2", top_k=3, confidence=0.3),
            SearchIndexRequest(index_name="summary-all-distilroberta-v1", top_k=3,confidence=0.3),
            SearchIndexRequest(index_name="summary-nomic-embed-text-v2", top_k=3, confidence=0.6),
        ],
        top_k=20,
    )
    
    # query generation test with all-MiniLM-L6-v2
    requests[f"{query_key}-query-gen"] = SearchRequest(
        query=query,
        generate_queries=True,
        index_requests=[
            SearchIndexRequest(index_name="all-MiniLM-L6-v2", top_k=20, confidence=0.0),
        ],
        top_k=20,
    )

In [17]:
responses : Dict[str, Dict] = {}
for key, request in requests.items():
    search_response: SearchResponse = search_service.search(request)
    if len(request.index_requests) == 1 and not request.generate_queries:
        del search_response.metrics["total_merge_time"] 
    if not request.generate_queries:
        del search_response.metrics["query_generation_time"]
        
    responses[key] = search_response.metrics

Model response: {
    "queries": [
        "How does Angular improve system performance by improving incremental hydration?",
        "What is the benefit of incremental hydration in Angular?",
        "How does Angular enhance system performance through incremental hydration?"
    ]
}
From How does incremental hydration help improve system performance? Generated queries: ['How does Angular improve system performance by improving incremental hydration?', 'What is the benefit of incremental hydration in Angular?', 'How does Angular enhance system performance through incremental hydration?']
Model response: {
    "queries": [
        "Angularjs: preventing cross site scriptings",
        "angularjs best practices for preventing XSS",
        "how to prevent cross site scripts in angular"
    ]
}
From Preventing cross-site scripting Generated queries: ['Angularjs: preventing cross site scriptings', 'angularjs best practices for preventing XSS', 'how to prevent cross site scripts in angula

In [18]:
output_file = "responses.json"
with open(output_file, "w") as file:
    json.dump(responses, file, indent=4)
print(f"Responses have been written to {output_file}")

Responses have been written to responses.json
